Experimento:
Buscamos lograr contestar la pregunta:

¿Es posible asociar el valor de alguna característica de la música como su energía o su tempo con un lugar geográfico?


In [50]:
import pandas as pd
import numpy as np


#juntar paises en base al continente
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [51]:
df = pd.read_excel('C:/Users/diego/OneDrive/Escritorio/universidad/semestre10/mineria/T2/Insight_Alchemist/Spotify.xlsx')

#Ajustamos columnas para contraarrestar error de formato en archivo xlsx
df['duration_ms'] = df['duration_ms']/10
df['popularity'] = df['popularity']/10
df['streams'] = df['streams']/10
df['af_danceability'] = df['af_danceability']/1000
df['af_energy'] = df['af_energy']/1000
df['af_key'] = df['af_key']/10
df['af_loudness'] = df['af_loudness']/1000
df['af_speechiness'] = df['af_speechiness']/1000
df['af_acousticness'] = df['af_acousticness']/1000
df['af_instrumentalness'] = df['af_instrumentalness']/1000
df['af_liveness'] = df['af_liveness']/1000
df['af_valence'] = df['af_valence']/1000
df['af_tempo'] = df['af_tempo']/1000
df['af_time_signature'] = df['af_time_signature']/10

Primero modificamos la información de la columna regiones para reducir las clases al agrupar los países por regiones geográficas más amplia.

In [52]:
d1 = [
    (['Morocco','South Africa'],'Africa'),
    (['Argentina','Bolivia','Brazil','Chile','Colombia','Costa Rica','Dominican Republic','Ecuador','El Salvador','Guatemala','Honduras','Mexico','Nicaragua','Panama','Paraguay','Peru','Uruguay'],'America_latina'),
    (['Canada','United States'],'America_norte'),
    (['Hong Kong','India','Indonesia','Japan','Malaysia','Philippines','Singapore','Taiwan','Thailand','Vietnam'],'Asia'),
    (['Andorra','Austria','Belgium','Bulgaria','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Iceland','Ireland','Italy','Latvia','Lithuania','Luxembourg',
     'Netherlands','Norway','Poland','Portugal','Romania','Russia','Slovakia','Spain','Sweden','Switzerland','Ukraine','United Kingdom'],'Europa'),
    (['Australia','New Zealand'],'Oceania'),
    (['Egypt','Israel','Saudi Arabia','Turkey','United Arab Emirates'],'Oriente_medio')
]

def apply_etiqueta(elemento):
    for i in d1:
        if elemento in i[0]:
            return i[1]
        
new_reg = df["region"]


for index, row in df.iterrows():
    new_data = apply_etiqueta(row["region"])
    new_reg.iat[index] = new_data

#se reemplaza la columa de region
df["region"] = new_reg


Para encontrar si esta relación existe vamos a usar un Decision Tree Classifier.

Primero vamos atributo por atributo probando para ver cual tiene mejores resultados.

In [53]:
for i in ['af_danceability','af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness','af_acousticness', 'af_liveness', 'af_valence','af_tempo', 'af_time_signature']:

    df = df.dropna(subset=["region", i])  # Drop rows where target or feature is NaN
    X = df[i].copy().values.reshape(-1, 1)
    y = df["region"]

    #primero separamos los datos de entrenamiento y validacion/test
    X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

    # Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
    X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

    #estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
    std_scaler = StandardScaler()
    X_train_std_scaled = std_scaler.fit_transform(X_train)

    X_val_std_scaled = std_scaler.transform(X_val)
    X_test_std_scaled = std_scaler.transform(X_test)

    #definimos nuestro modelo
    clf = DecisionTreeClassifier(criterion="gini",random_state=0)

    #entrenamos al modelo con fit
    clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

    #realizamos predicciones de nuestros datos con los datos de validación
    y_val_pred = clf.predict(X_val_std_scaled)


    #metricas de las predicciones
    #print(i)
    #print(classification_report(y_val, y_val_pred))

De lo anterior se pude observar que las dos características que presentan mayor relación son loudness y tempo.

Ahora se va a hacer un codigo que pruebe con todas las posibilidades

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

L = ['af_danceability','af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness','af_acousticness', 'af_liveness', 'af_valence','af_tempo', 'af_time_signature']

# Se crea una lista con todas las combinaciónes posibles
print(len(L))
newList = []
def sublists(lst, new, index=0, current=[]):
    if index == len(lst):
        new.append(current)
        #print(current)
        return
    sublists(lst,new, index+1, current)
    sublists(lst,new, index+1, current + [lst[index]])

sublists(L,newList)

print(len(newList))
#sacamos los repetidos
unique_newList = []
for i in newList:
    if i not in unique_newList:
        unique_newList.append(i)

#sacamos el elemento vacio
unique_newList.remove([])
print(len(unique_newList))

#accuracy list
accuracy_list = []

for i in unique_newList:
    subset = ["region"] + i
    df = df.dropna(subset=subset)  # Drop rows where target or feature is NaN
    X = df[i].copy()
    y = df["region"]

    #primero separamos los datos de entrenamiento y validacion/test
    X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

    # Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
    X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

    #estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
    std_scaler = StandardScaler()
    X_train_std_scaled = std_scaler.fit_transform(X_train)

    X_val_std_scaled = std_scaler.transform(X_val)
    X_test_std_scaled = std_scaler.transform(X_test)

    #definimos nuestro modelo
    clf = DecisionTreeClassifier(criterion="gini",random_state=0)

    #entrenamos al modelo con fit
    clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

    #realizamos predicciones de nuestros datos con los datos de validación
    y_val_pred = clf.predict(X_val_std_scaled)


    #metricas de las predicciones

    accuracy = accuracy_score(y_val,y_val_pred)
    precision_micro = precision_score(y_val, y_val_pred, average='micro')
    precision_macro = precision_score(y_val, y_val_pred, average='macro')
    recall_micro = recall_score(y_val, y_val_pred, average='micro')
    recall_macro = recall_score(y_val, y_val_pred, average='macro')
    f1_micro = f1_score(y_val, y_val_pred, average='micro')
    f1_macro = f1_score(y_val, y_val_pred, average='macro')

    #para poder saber cual fue el mejor se va a sacar el promedio de todos los score y se van a guardar en una lista
    promedio = (accuracy + precision_micro + precision_macro + recall_micro + recall_macro + f1_micro + f1_macro)/7 

    accuracy_list.append((i,promedio))

print(accuracy_list)
#ordenamos la lista resultante
sorted_by_score = sorted(accuracy_list, key=lambda tup:tup[1])


11
2048
2047
0


c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


64


...

257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319


c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


320


c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570


Se imprimen las 10 peores y las 10 mejores conjuntos

In [59]:
print(sorted_by_score[:10])
print(sorted_by_score[2036:2046])

[(['af_time_signature'], np.float64(0.2864815037161335)), (['af_mode'], np.float64(0.2864815037161335)), (['af_mode', 'af_time_signature'], np.float64(0.2864815037161335)), (['af_key'], np.float64(0.2864815037161335)), (['af_key', 'af_mode'], np.float64(0.29640393600964965)), (['af_key', 'af_time_signature'], np.float64(0.31580236121712757)), (['af_key', 'af_mode', 'af_time_signature'], np.float64(0.33895782009387393)), (['af_liveness'], np.float64(0.4260734008648214)), (['af_acousticness'], np.float64(0.4377899173040535)), (['af_danceability'], np.float64(0.43925500120213584))]
[(['af_key', 'af_liveness', 'af_valence', 'af_tempo', 'af_time_signature'], np.float64(0.7016625443436151)), (['af_energy', 'af_key', 'af_loudness', 'af_acousticness', 'af_valence', 'af_tempo', 'af_time_signature'], np.float64(0.7016709033736281)), (['af_danceability', 'af_energy', 'af_key', 'af_speechiness', 'af_acousticness', 'af_valence', 'af_tempo'], np.float64(0.7016783483969826)), (['af_key', 'af_mode', '

Se predice usando los datos test con el mejor conjunto

In [61]:
df = df.dropna(subset=['region','af_danceability', 'af_energy', 'af_mode', 'af_speechiness', 'af_acousticness', 'af_valence', 'af_tempo'])  # Drop rows where target or feature is NaN
X = df[['af_danceability', 'af_energy', 'af_mode', 'af_speechiness', 'af_acousticness', 'af_valence', 'af_tempo']].copy()
y = df["region"]

#primero separamos los datos de entrenamiento y validacion/test
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

# Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

#estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
std_scaler = StandardScaler()
X_train_std_scaled = std_scaler.fit_transform(X_train)

X_val_std_scaled = std_scaler.transform(X_val)
X_test_std_scaled = std_scaler.transform(X_test)

#definimos nuestro modelo
clf = DecisionTreeClassifier(criterion="gini",random_state=0)

#entrenamos al modelo con fit
clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

#realizamos predicciones de nuestros datos con los datos de validación
y_test_pred = clf.predict(X_test_std_scaled)


#metricas de las predicciones
print("todas")
print(classification_report(y_test, y_test_pred))


todas
                precision    recall  f1-score   support

        Africa       0.93      0.46      0.62      1785
America_latina       0.82      0.75      0.79     15151
 America_norte       0.64      0.17      0.27      1785
          Asia       0.88      0.67      0.76      8922
        Europa       0.68      0.92      0.78     25448
       Oceania       0.83      0.22      0.35      1785
 Oriente_medio       0.86      0.38      0.53      4461

      accuracy                           0.74     59337
     macro avg       0.80      0.51      0.58     59337
  weighted avg       0.77      0.74      0.73     59337

